In [3]:
from datetime import datetime, timedelta
import os
import json
import requests
import hmac
import pandas as pd
import pandas_datareader as pdr
import datetime as dt
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as st
from scipy.stats import linregress
import time
from config import API_KEY
from client import FtxClient

ModuleNotFoundError: No module named 'config'

In [4]:
url = 'https://ftx.com/api/markets'

#get BTCUSD spot price
base_currency = 'BTC'
quote_currency = 'USD'

#construct request url
request_url = f"{url}/{base_currency}/{quote_currency}"

btcusd_df = pd.DataFrame(requests.get(request_url).json())
btcusd_df['result']



ask                             24591.0
baseCurrency                        BTC
bid                             24590.0
change1h                      -0.000975
change24h                     -0.003808
changeBod                       0.00622
enabled                            True
futureType                         None
highLeverageFeeExempt              True
isEtfMarket                       False
largeOrderThreshold              3000.0
last                            24591.0
minProvideSize                   0.0001
name                            BTC/USD
postOnly                          False
price                           24591.0
priceHigh24h                    24900.0
priceIncrement                      1.0
priceLow24h                     24290.0
quoteCurrency                       USD
quoteVolume24h           324935351.9612
restricted                        False
sizeIncrement                    0.0001
type                               spot
underlying                         None


In [5]:
def historical_bymonth(y, m, res="hourly"):
    """grabs historical BTCUSD 1H price from the FTX API by individual month

    Args:
        y (int): year
        m (int): month
        res (str): resolution options "hourly" or "daily", default is "hourly"
    """
    
    resolution = {
        
        "hourly":str(60*60),
        "daily":str(60*60*24)
    }
    
    
    start_date = datetime(year=y, month=m, day=1).timestamp()
    #end date wraps around across years
    end_date = (datetime(y if m<12 else y+1, (m % 12)+1, 1) - timedelta(seconds=15)).timestamp()
    
    historical = requests.get(
                f'{request_url}/candles?resolution={resolution[res]}&start_time={start_date}&end_time={end_date}'
                ).json()
    
    ls = historical['result']
    # df = pd.DataFrame(historical['result'])
    
    return ls

def get_multiple_months(y,m,n_months):
    """grabs multiple months of historical 1 hour data for BTCUSD FTX

    Args:
        y (int): starting year
        m (month): starting month
        n_months (int): number of months of data to be retrieved from the designated starting date
    """
    
    #maximum of 12 queries per second according to FTX docs
    MAX_PING_PER_SEC = 12
    
    ls = []
    
    
    for _ in range(n_months):
        ls.extend(historical_bymonth(y, m))
        
        if m % 12 == 0:
            y+=1
            
        m = (m % 12)+1

        #staggers every 1/12 seconds so that limit error cannot be triggered for any n_months requested
        time.sleep(1.0 / float(MAX_PING_PER_SEC))
    
    return ls

ls = get_multiple_months(2021,1,20)
df = pd.DataFrame(ls)
df

#convert date/time units
df['date'] = pd.to_datetime(
    df['time']/1000, unit='s', origin='unix'
)

#remove unnecessary columns
df.drop(['startTime','time'], axis=1, inplace=True)

df    
       
        

# df2 = historical_bymonth(2021,6,res="thirty")
# df2
# y = 2021
# m = 12
# print(datetime(y if m<12 else y+1, (m % 12)+1, 1))
# print(datetime(y if m<12 else y+1, (m % 12)+1, 1) - timedelta(microseconds=1))
# print((datetime(y if m<12 else y+1, (m % 12)+1, 1) - timedelta(microseconds=1)).timestamp())
    

,open,high,low,close,volume,date
0,29280.5,29296.0,29146.5,29240.0,5.523795e+06,2021-01-01 05:00:00
1,29240.0,29315.0,29127.5,29231.0,7.436047e+06,2021-01-01 06:00:00
2,29231.0,29241.5,28887.0,29168.0,8.430755e+06,2021-01-01 07:00:00
3,29168.0,29251.0,28928.5,29061.0,7.003067e+06,2021-01-01 08:00:00
4,29061.0,29379.5,29041.5,29275.0,5.764370e+06,2021-01-01 09:00:00
...,...,...,...,...,...,...
14154,24390.0,24504.0,24368.0,24447.0,1.217444e+07,2022-08-13 23:00:00
14155,24447.0,24521.0,24350.0,24520.0,8.810785e+06,2022-08-14 00:00:00
14156,24520.0,24645.0,24417.0,24597.0,1.683024e+07,2022-08-14 01:00:00
14157,24597.0,24625.0,24508.0,24612.0,8.566078e+06,2022-08-14 02:00:00


In [6]:
#return SMA values
def sma(df, window=9):
    return df.rolling(window=window).mean()

#return EMA values
def ema(df, window=9):
    return df.ewm(span=window, adjust=False).mean()


def liq(direction, lev=25, maint = 0.005):
    """Calculates liquidation price given the entry price (EMA9 is used here to track price over average of ~1 session where one session = 8hrs
    #Liquidation calculation provided by exchange
    Liquidation Price = ((Price*Leverage)/(Leverage+1 - (Maintenance Margin * Leverage))) 
    

    Args:
        lev (int): leverage used (options: 10,25,50,100)
        direction (string): 'long' or 'short' position
        maint (float): maintanance margin (0.005 for BTCUSD and 0.01 for ETHUSD)
        

    Returns:
        ls: list of liquidation prices 
    """
    ls = []
    ema9 = df['EMA9']
    
    if maint == 0.005:
        if direction == 'long':
            
            if lev == 10:
                ls = ((ema9*10)/(10+1-(0.005*10)))
            elif lev == 25:
                ls = ((ema9*25)/(25+1-(0.005*25)))
            elif lev == 50:
                ls = ((ema9*50)/(50+1-(0.005*50)))
            else:
                ls = ((ema9*99)/(99+1-(0.005*99)))
            
        else:
            
            if lev == 10:
                ls = ((ema9*10)/(10-1+(0.005*10)))
            elif lev == 25:
                ls = ((ema9*25)/(25-1+(0.005*25)))
            elif lev == 50:
                ls = ((ema9*50)/(50-1+(0.005*50)))
            else:
                ls = ((ema9*99)/(99-1+(0.005*99)))
    else:
        pass
                
    
    
    return ls

#add ohlc4 column
df['ohlc4'] = (df['open']+df['high']+df['low']+df['close'])/4

#add SMA column 
df['SMA48'] = sma(df['ohlc4'], window=48)
df['SMA9'] = sma(df['ohlc4'], window=9)


#add EMA columns
df['EMA9'] = ema(df['ohlc4'], window=9)
df['EMA48'] = ema(df['ohlc4'], window=48)
ema9 = df['EMA9']


# add liquidation value columns 10,25,50,100x 
df['10x short'] = liq('short',10)
df['25x short'] = liq('short',25)
df['50x short'] = liq('short',50)
df['100x short'] = liq('short',100)

df['10x long'] = liq('long',10)
df['25x long'] = liq('long',25)
df['50x long'] = liq('long',50)
df['100x long'] = liq('long',100)


df 
data = df


In [7]:
#create hour column
df['hour']=df.date.dt.hour
df['hour']=df['hour'].astype(float)

In [8]:

#date for volatility
data = df.set_index('date')

high_low = data['high']-data['low']
high_cp = np.abs(data['high']-data['close'].shift())
low_cp = np.abs(data['low']-data['close'].shift())

df_volatility = pd.concat([high_low, high_cp, low_cp], axis =1)

#calc true range
true_range = np.max(df_volatility, axis =1)

#calc avg true range
average_true_range = true_range.rolling(14).mean()

data['average_true_range']=average_true_range


data = data.reset_index(drop=False)

data = data.rename(columns = {"volume":"volume(BTC)"})

data = data[['date','hour','open','high','low','close','volume(BTC)','ohlc4', 'SMA48', 'average_true_range',
             'EMA9','EMA48','10x short','25x short', '50x short', '100x short', '10x long', '25x long', '50x long','100x long']]
data

,date,hour,open,high,low,close,volume(BTC),ohlc4,SMA48,average_true_range,EMA9,EMA48,10x short,25x short,50x short,100x short,10x long,25x long,50x long,100x long
0,2021-01-01 05:00:00,5.0,29280.5,29296.0,29146.5,29240.0,5.523795e+06,29240.750,NaN,NaN,29240.750000,29240.750000,32310.220994,30301.295337,29686.040609,29390.672115,26703.881279,28251.932367,28808.620690,29092.349631
1,2021-01-01 06:00:00,6.0,29240.0,29315.0,29127.5,29231.0,7.436047e+06,29228.375,NaN,NaN,29238.275000,29240.244898,32307.486188,30298.730570,29683.527919,29388.184426,26701.621005,28249.541063,28806.182266,29089.887192
2,2021-01-01 07:00:00,7.0,29231.0,29241.5,28887.0,29168.0,8.430755e+06,29131.875,NaN,NaN,29216.995000,29235.821637,32283.972376,30276.678756,29661.923858,29366.795320,26682.187215,28228.980676,28785.216749,29068.715190
3,2021-01-01 08:00:00,8.0,29168.0,29251.0,28928.5,29061.0,7.003067e+06,29102.125,NaN,NaN,29194.021000,29230.364631,32258.586740,30252.871503,29638.600000,29343.703528,26661.206393,28206.783575,28762.582266,29045.857786
4,2021-01-01 09:00:00,9.0,29061.0,29379.5,29041.5,29275.0,5.764370e+06,29189.250,NaN,NaN,29193.066800,29228.686483,32257.532376,30251.882694,29637.631269,29342.744436,26660.334977,28205.861643,28761.642167,29044.908429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14154,2022-08-13 23:00:00,23.0,24390.0,24504.0,24368.0,24447.0,1.217444e+07,24427.250,24263.130208,148.500000,24468.452935,24336.943555,27036.964569,25355.909777,24841.068969,24593.906701,22345.619119,23641.017328,24106.850182,24344.272555
14155,2022-08-14 00:00:00,0.0,24447.0,24521.0,24350.0,24520.0,8.810785e+06,24459.500,24274.171875,147.357143,24466.662348,24341.945859,27034.986020,25354.054246,24839.251115,24592.106934,22343.983879,23639.287293,24105.086057,24342.491055
14156,2022-08-14 01:00:00,1.0,24520.0,24645.0,24417.0,24597.0,1.683024e+07,24544.750,24286.687500,147.357143,24482.279878,24350.223579,27052.242959,25370.238216,24855.106475,24607.804538,22358.246464,23654.376694,24120.472786,24358.029325
14157,2022-08-14 02:00:00,2.0,24597.0,24625.0,24508.0,24612.0,8.566078e+06,24585.500,24299.656250,143.714286,24502.923903,24359.826698,27075.054036,25391.630987,24876.064876,24628.554407,22377.099454,23674.322611,24140.811727,24378.568578


In [9]:
cols = ['ohlc4', 'SMA48', 'average_true_range','EMA9','EMA48','10x short','25x short', '50x short', '100x short', '10x long', '25x long', '50x long','100x long']
data[cols] = data[cols].round(2)
data

,date,hour,open,high,low,close,volume(BTC),ohlc4,SMA48,average_true_range,EMA9,EMA48,10x short,25x short,50x short,100x short,10x long,25x long,50x long,100x long
0,2021-01-01 05:00:00,5.0,29280.5,29296.0,29146.5,29240.0,5.523795e+06,29240.75,NaN,NaN,29240.75,29240.75,32310.22,30301.30,29686.04,29390.67,26703.88,28251.93,28808.62,29092.35
1,2021-01-01 06:00:00,6.0,29240.0,29315.0,29127.5,29231.0,7.436047e+06,29228.38,NaN,NaN,29238.28,29240.24,32307.49,30298.73,29683.53,29388.18,26701.62,28249.54,28806.18,29089.89
2,2021-01-01 07:00:00,7.0,29231.0,29241.5,28887.0,29168.0,8.430755e+06,29131.88,NaN,NaN,29217.00,29235.82,32283.97,30276.68,29661.92,29366.80,26682.19,28228.98,28785.22,29068.72
3,2021-01-01 08:00:00,8.0,29168.0,29251.0,28928.5,29061.0,7.003067e+06,29102.12,NaN,NaN,29194.02,29230.36,32258.59,30252.87,29638.60,29343.70,26661.21,28206.78,28762.58,29045.86
4,2021-01-01 09:00:00,9.0,29061.0,29379.5,29041.5,29275.0,5.764370e+06,29189.25,NaN,NaN,29193.07,29228.69,32257.53,30251.88,29637.63,29342.74,26660.33,28205.86,28761.64,29044.91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14154,2022-08-13 23:00:00,23.0,24390.0,24504.0,24368.0,24447.0,1.217444e+07,24427.25,24263.13,148.50,24468.45,24336.94,27036.96,25355.91,24841.07,24593.91,22345.62,23641.02,24106.85,24344.27
14155,2022-08-14 00:00:00,0.0,24447.0,24521.0,24350.0,24520.0,8.810785e+06,24459.50,24274.17,147.36,24466.66,24341.95,27034.99,25354.05,24839.25,24592.11,22343.98,23639.29,24105.09,24342.49
14156,2022-08-14 01:00:00,1.0,24520.0,24645.0,24417.0,24597.0,1.683024e+07,24544.75,24286.69,147.36,24482.28,24350.22,27052.24,25370.24,24855.11,24607.80,22358.25,23654.38,24120.47,24358.03
14157,2022-08-14 02:00:00,2.0,24597.0,24625.0,24508.0,24612.0,8.566078e+06,24585.50,24299.66,143.71,24502.92,24359.83,27075.05,25391.63,24876.06,24628.55,22377.10,23674.32,24140.81,24378.57


In [11]:
#save as csv
data.to_csv("resources/2021_2022_FTX_USDBTC_1H.csv", index=False, header=True)